In [8]:
import bibtexparser
import pandas as pd
import requests
from lxml.html import fromstring
import re
import sys
import unicodedata

tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))

Getting lists of journals and conferences (conferences not used at the moment) and filter them based on topic codes (has to include at least one of the ones in scopus_dict and not contain one of those in scopus_not). The list for web of science was scraped, scopus and ebsco are from their websites

In [2]:
scopus_conferences = pd.read_csv('Conference_Proceedings_list.csv', sep = ';')[['ID', 'Title','Code', 'Code1', 'Code2', 'Code3', 'Code4']]

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
scopus_dict = {"1702":"Artificial Intelligence", "3315":"Communication", "1705":"Computer Networks and Communications", "1706":"Computer Science Applications",\
               "1709":"Human-Computer Interaction", "1710":"Information Systems","1203":"Language and Linguistics", "3308":"Law",\
               "3309":"Library and Information Sciences", "3310":"Linguistics and Language","2214":"Media Technology","3320":"Political Science and International Relations",\
               "3207":"Social Psychology","3312":"Sociology and Political Science"}

In [4]:
scopus_list = list(scopus_dict.keys())

In [5]:
scopus_not = list(map(str,[1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1600, 1601, 1602, 1603, 1604, 1605, 1606, 1607, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 2100, 2101, 2102, 2103, 2104, 2105, 2202, 2203, 2204, 2209, 2210, 2211, 2212, 2213, 2215, 2216, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2309, 2310, 2311, 2312, 2400, 2401, 2402, 2403, 2404, 2405, 2406, 2500, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2608, 2610, 2700, 2701, 2702, 2703, 2704, 2705, 2706, 2707, 2708, 2709, 2710, 2711, 2712, 2713, 2714, 2715, 2716, 2717, 2718, 2719, 2720, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733, 2734, 2735, 2736, 2737, 2738, 2739, 2740, 2741, 2742, 2743, 2744, 2745, 2746, 2747, 2748, 2800, 2801, 2802, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2900, 2901, 2902, 2903, 2904, 2905, 2906, 2907, 2908, 2909, 2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919, 2920, 2921, 2922, 2923, 3000, 3001, 3002, 3003, 3004, 3005, 3100, 3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109, 3110, 3203, 3204, 3205, 3206, 3302, 3303, 3304, 3305, 3306, 3307, 3311, 3313, 3314, 3318, 3319, 3322, 3400, 3401, 3402, 3403, 3404, 3500, 3501, 3502, 3503, 3504, 3505, 3506, 3600, 3601, 3602, 3603, 3604, 3605, 3606, 3607, 3608, 3609, 3610, 3611, 3612, 3613, 3614, 3615, 3616]))

In [6]:
len(scopus_conferences)

29683

In [7]:
scopus_conferences = scopus_conferences[(scopus_conferences.Code.isin(scopus_list)|scopus_conferences.Code1.isin(scopus_list)|scopus_conferences.Code2.isin(scopus_list)|scopus_conferences.Code3.isin(scopus_list)|scopus_conferences.Code4.isin(scopus_list))&((~scopus_conferences['Code'].isin(scopus_not))&(~scopus_conferences['Code1'].isin(scopus_not))&(~scopus_conferences['Code2'].isin(scopus_not))&(~scopus_conferences['Code3'].isin(scopus_not))&(~scopus_conferences['Code4'].isin(scopus_not)))]

In [8]:
len(scopus_conferences)

9994

In [9]:
scopus_conferences = scopus_conferences['Title'].str.replace('{\&}', 'and', regex = False).str.lower().str.replace('&', 'and', regex = False).str.translate(tbl).tolist()

In [10]:
scopus_journals = pd.read_csv('scopus_journals.csv', sep = ';')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (4,5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
scopus_journals['title_wos'] = scopus_journals['Title'].str.replace('{\&}', 'and', regex = False).str.lower().str.replace('&', 'and', regex = False).str.translate(tbl)

In [12]:
scopus_journals = scopus_journals[(scopus_journals.Code.isin(scopus_list)|scopus_journals.Code1.isin(scopus_list)|scopus_journals.Code2.isin(scopus_list)|scopus_journals.Code3.isin(scopus_list)|scopus_journals.Code4.isin(scopus_list)|scopus_journals.Code5.isin(scopus_list)|scopus_journals.Code6.isin(scopus_list))&((~scopus_journals['Code'].isin(scopus_not))&(~scopus_journals['Code1'].isin(scopus_not))&(~scopus_journals['Code2'].isin(scopus_not))&(~scopus_journals['Code3'].isin(scopus_not))&(~scopus_journals['Code4'].isin(scopus_not))&(~scopus_journals['Code5'].isin(scopus_not))|(~scopus_journals['Code6'].isin(scopus_not)))]

In [13]:
scopus_journals = scopus_journals['Title'].str.replace('{\&}', 'and', regex = False).str.lower().str.replace('&', 'and', regex = False).str.translate(tbl).tolist()

In [14]:
len(scopus_journals)

4547

In [14]:
#base_url = 'http://mjl.clarivate.com/cgi-bin/jrnlst/jlresults.cgi?PC='
#end = '&mode=print&Page='
#letter_numbers = [('H', list(range(1,5))),('D', list(range(1,20))),('SS', list(range(1,8))),('EX', list(range(1,17)))]
#wos_journals = []
#for item in letter_numbers:
#    for n in item[1]:
#        url = base_url + item[0] + end + str(n)
#        website = requests.get(url, timeout = 10)
#        tree = fromstring(website.text)
#        journals = (tree.xpath('.//strong/text()'))[2:]
#        journals = [re.sub(r'\d{1,4}[.] ', '', item) for item in journals]
#        journals = [re.sub(r"&", 'and', item) for item in journals]
#        journals = [item.lower().translate(tbl) for item in journals]  
#        wos_journals.extend(journals)
#wos_journals = pd.Series(wos_journals)

In [15]:
#wos_journals.to_csv('wos_journals.csv')

In [15]:
wos_journals = pd.read_csv('wos_journals.csv')

In [16]:
wos_not_scopus = wos_journals[~wos_journals.isin(scopus_journals.title_wos)]

AttributeError: 'list' object has no attribute 'title_wos'

Getting the bibtex files downloaded from the databases as Python dicts

In [17]:
with open('scopus.bib') as bibtex_file:
    bib_database_scopus = bibtexparser.load(bibtex_file)
with open('wos.bib') as bibtex_file:
    bib_database_wos = bibtexparser.load(bibtex_file)
with open('ebsco.bib') as bibtex_file:
    bib_database_ebsco = bibtexparser.load(bibtex_file)

Not included at the moment: arXiv

In [18]:
#results = arxiv.query('(abs:divers* OR abs:"filter bubble*" OR abs:"echo chamber*" OR abs:serendip* OR abs:plural* OR abs:"entropy") AND (abs:news* OR abs:"media" OR abs:Twitter OR abs:Facebook OR abs:reddit OR abs:journalis* OR abs:"social media") AND (cat:cs.AI OR cat:cs.CL OR cat:cs.CY OR cat:cs.DS OR cat:cs.HC OR cat:cs.IR OR cat:cs.LG OR cat:cs.SI)', max_results = 1000)

In [19]:
#len(results)

In [20]:
#bib_database_arxiv = []
#for item in results: 
#    authors = ' and '.join(item['authors'])
#    entry = {'author':authors,
#             'outlet':"ARXIV",
#             'ENTRYTYPE':"inproceedings",
#             'abstract':item['summary'].replace('\n', ' '),
#             'url':item['pdf_url'],
#             'doi':item['doi'],
#             'title':item['title'],
#             'year':item['published'][:4]
#            }
#    bib_database_arxiv.append(entry)


Putting files into pandas dataframes and only select articles (not including conference proceedings for now)

In [21]:
wos = pd.DataFrame(bib_database_wos.entries)
scopus = pd.DataFrame(bib_database_scopus.entries)
ebsco = pd.DataFrame(bib_database_ebsco.entries)
#arxiv = pd.DataFrame(bib_database_arxiv)

In [29]:
#arxiv['title_full'] = arxiv['title']
#arxiv['title'] = arxiv['title'].str.replace('{\&}', 'and', regex = False).str.replace('&', 'and', regex = False).str.lower().str.translate(tbl)
#arxiv['database'] = 'arxiv'

In [30]:
scopus = scopus[scopus['ENTRYTYPE'] == 'article']
wos = wos[wos['ENTRYTYPE'] == 'article']
ebsco = ebsco[ebsco['ENTRYTYPE'] == 'article']

In [32]:
print(len(ebsco), len(wos), len(scopus), len(ebsco) + len(wos) + len(scopus))

2010 2177 11449 15636


Putting all outlet fields into one column, using the same processing steps as for the journal lists (lowercasing, replacing special characters) to account for differences in spelling; adding a database column

In [33]:
wos['outlet'] = wos['journal'].combine_first(wos['booktitle'])
wos['outlet'] = wos['outlet'].combine_first(wos['series'])
wos['outlet_low'] = wos['outlet'].str.replace('{\&}', 'and', regex = False).str.lower().str.replace('&', 'and', regex = False).str.translate(tbl)
wos['title_full'] = wos['title']
wos['title'] = wos['title'].str.replace('{\&}', 'and', regex = False).str.replace('&', 'and', regex = False).str.lower().str.translate(tbl)
wos['database'] = 'wos'
scopus['outlet'] = scopus['journal'].combine_first(scopus['booktitle'])
scopus['outlet_low'] = scopus['outlet'].str.replace('{\&}', 'and', regex = False).str.replace('&', 'and', regex = False).str.lower().str.translate(tbl)
scopus['title_full'] = scopus['title']
scopus['title'] = scopus['title'].str.replace('{\&}', 'and', regex = False).str.replace('&', 'and', regex = False).str.lower().str.translate(tbl)
scopus['url'] = scopus['url'].str.replace('{\&}', '&', regex = False).str.replace('{\%}', '%', regex = False)
scopus['database'] = 'scopus'
ebsco['outlet'] = ebsco['journal']
ebsco['outlet_low'] = ebsco['journal'].str.replace('{\&}', 'and', regex = False).str.replace('&', 'and', regex = False).str.lower().str.translate(tbl)
ebsco['title_full'] = ebsco['title']
ebsco['title'] = ebsco['title'].str.replace('{\&}', 'and', regex = False).str.replace('&', 'and', regex = False).str.lower().str.translate(tbl)
ebsco['database'] = 'ebsco'

In [34]:
wos_journals = list(dict(wos[wos.outlet_low.isin(wos_not_scopus)]['outlet_low'].value_counts()).keys())

For Ebsco the title-list was manually inspected and Journals that are not of interest (i.e. sociological, gender studies, non-English journals) are removed. This leaves 57 titles of interest from EBSCO

In [35]:
titles_ebsco = pd.read_csv("ebsco_journals.csv", sep = ";")['Publication Name'].tolist()
titles_ebsco = [i.replace('&', 'and').lower().translate(tbl) for i in titles_ebsco]
len(titles_ebsco)

57

In [36]:
print(len(wos_journals) + len(scopus_journals) + len(titles_ebsco))

4604


In [37]:
wos = wos.loc[wos['outlet_low'].isin(scopus_journals) | wos['outlet_low'].isin(wos_journals) | wos['outlet_low'].isin(scopus_conferences)]
scopus = scopus.loc[scopus['outlet_low'].isin(scopus_journals)| wos['outlet_low'].isin(scopus_conferences)]
ebsco = ebsco.loc[ebsco['outlet_low'].isin(titles_ebsco)| (ebsco['ENTRYTYPE'] == 'inproceedings') | (ebsco['ENTRYTYPE'] == 'conference')]

In [38]:
print(len(ebsco),len(wos),len(scopus),len(ebsco) + len(wos) + len(scopus))

564 1778 6449 8791


In [22]:
all_data = scopus.append(wos, ignore_index = True, sort = True)

In [23]:
all_data = all_data.append(ebsco, ignore_index = True, sort = True)

In [26]:
all_data.to_csv('all_references.csv')

Removing duplicates based on doi and title

In [41]:
all_data = all_data[(~all_data.duplicated(['doi'])) | (all_data['doi'].isnull())]

In [42]:
len(all_data)

7746

In [43]:
all_data = all_data.drop_duplicates(['title'], keep = 'first')

In [44]:
len(all_data)

7309

In [45]:
all_data['database'].value_counts()

scopus    6440
wos        633
ebsco      236
Name: database, dtype: int64

Extracting IDs for ebsco and web of science (make finding articles automatically in the databases easier since those two databases do not always give a doi)

In [46]:
def make_ebsco_id (string):
    try:
        if 'ebscohost' in string['url']:
            if re.search(r'AN=(.*?){', string['url']):
                ebsco_id = re.search(r'AN=(.*?){', string['url']).group(1)
            else:
                ebsco_id = re.search(r'AN=(.*?)&', string['url']).group(1)
        else:
            ebsco_id = ""
    except:
        ebsco_id = ""
    return ebsco_id

In [47]:
def make_wos_id (string):
    try:
        if 'ISI' in string['ID']:
            wos_id = string['ID'][4:]
        else:
            wos_id = ""
    except:
        wos_id = ""
    return wos_id

In [48]:
all_data['ebsco_id'] = all_data.apply(make_ebsco_id, axis=1)

In [49]:
all_data['wos_id'] = all_data.apply(make_wos_id, axis=1)

In [50]:
all_data = all_data.applymap(str)

Using regular expressions to filter the data again based on the original search string. Additionally also including the results that were only found because of "Springer Media".

In [51]:
all_data['abstract'] = all_data['abstract'].str.replace('\n',' ').str.replace('\r', ' ').str.replace('\\', ' ').str.replace('{', ' ').str.replace('}', ' ')

In [52]:
all_data = all_data[(all_data.abstract.str.contains('[Nn]ews|[Nn]ews+|media|[Jj]ournalis+|[Tt]witter|[Ff]acebook|[rR]eddit|social media|social network', regex = True))|(all_data.title.str.contains('[Nn]ews|[Nn]ews+|media|[Jj]ournalis+|[Tt]witter|[Ff]acebook|[rR]eddit|social media|social network', regex = True))|(all_data.keywords.str.contains('[Nn]ews|[Nn]ews+|media|[Jj]ournalis+|[Tt]witter|[Ff]acebook|[rR]eddit|social media|social network', regex = True))|(all_data['database']=='ebsco')]

In [53]:
all_data = all_data[(all_data.abstract.str.contains('[Dd]ivers+|[Ee]cho [Cc]hambe+|[Ff]ilter [Bb]ubbl+|[Ss]erendip+|[Pp]lurali+|entropy', regex = True))|(all_data.title.str.contains('[Dd]ivers+|[Ee]cho [Cc]hamb+|[Ff]ilter [Bb]ubbl+|[Ss]erendip+|[Pp]lurali+|entropy', regex = True))|(all_data.keywords.str.contains('[Dd]ivers+|[Ee]cho [Cc]hamb+|[Ff]ilter [Bb]ubbl+|[Ss]erendip+|[Pp]lurali+|entropy', regex = True))|(all_data['database']=='ebsco')]

In [54]:
springer_data = all_data[(all_data.abstract.str.contains('[Mm]edia', regex = True)) & (all_data.abstract.str.contains('[Ss]pringer', regex = True))&(~all_data.abstract.str.contains('[nN]ews+|[Jj]ournalis+|[Ff]acebook|[Tt]witter|[Rr]eddit', regex = True))]

In [55]:
all_data = all_data[~all_data.ID.isin(springer_data.ID)]

In [56]:
len(all_data)

3862

In [57]:
all_data['database'].value_counts()

scopus    3201
wos        425
ebsco      236
Name: database, dtype: int64

Transforming dataframe to dict and then to bibtex file to check duplicates in Mendeley (also for near-exact matches) 

In [59]:
all_data_bib = all_data.to_dict('records')

In [60]:
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase

db = BibDatabase()
db.entries = all_data_bib

writer = BibTexWriter()
with open('articles.bib', 'w') as bibfile:
    bibfile.write(writer.write(db))

Identify Near-Duplicates in Mendeley

In [61]:
duplicate_ids = ['https://www.scopus.com/inward/record.uri?eid=2-s2.0-84861543428&partnerID=40&md5=9d884393523f89474d2cf9420772db67','https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=127361920&site=ehost-live&scope=site','https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=122543490&site=ehost-live&scope=site', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=40623477&site=ehost-live&scope=site', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=40623478&site=ehost-live&scope=site', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=67071939&site=ehost-live&scope=site', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=96133769&site=ehost-live&scope=site', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=67071943&site=ehost-live&scope=site', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=45447006&site=ehost-live&scope=site', 'ISI:000361974400004', 'ISI:000391329800001', 'ISI:000276166100005', 'ISI:000361977300045', 'ISI:000408476100003', 'ISI:000361977300043', 'ISI:000389829700001', 'ISI:000386482100001', 'ISI:000402075800005', 'ISI:000276135000003', 'ISI:000326849300004', 'ISI:000377716600013', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=100099789&site=ehost-live&scope=site', 'ISI:000411096600019', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=25416763&site=ehost-live&scope=site', 'ISI:000391044400001', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=22585612&site=ehost-live&scope=site', 'https://search.ebscohost.com/login.aspx?direct=true&db=ufh&AN=126812983&site=ehost-live&scope=site', 'ISI:000226742800005']

In [62]:
all_data = all_data[(~all_data['url'].isin(duplicate_ids)) & (~all_data['ID'].isin(duplicate_ids))]

Making a CSV file for qualtrics. Additionally, a ID is created as well as an "email", making it possible to [import the CSV file into qualtrics as contacts](https://www.qualtrics.com/support/survey-platform/contacts/contact-list-overview/). This way, the keys are included as embedded data and do not need to be coded again and can be displayed to the coder using piped text. For every entry/contact an individual link is created to code the article. 

In [63]:
for_qualtrics = all_data[['ID', 'abstract', 'author', 'doi', 'ebsco_id', 'outlet', 'title_full', 'url', 'wos_id', 'year']]

In [64]:
for_qualtrics['qualtrics_id'] = for_qualtrics.index

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
for_qualtrics['qualtrics_id'] = for_qualtrics['qualtrics_id'].apply('{:0>6}'.format)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Make new IDS for the articles

In [66]:
def make_email(x):    
    return x['qualtrics_id'] + "@" + x['year'] + ".nl"

In [67]:
for_qualtrics['year'] = for_qualtrics.year.astype(str)
for_qualtrics['qualtrics_id'] = for_qualtrics.qualtrics_id.astype(str)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [68]:
for_qualtrics['Email'] = for_qualtrics.apply(make_email, axis=1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [69]:
for_qualtrics.to_csv('qualtrics_list.csv', index = False, sep = ',')

Last step (was done in excel) is to make the search words all CAPS in the abstracts so it is possible to see why an article was found by one look. Need to change seperator as excel is too stupid for that. 

In [ ]:
with_caps = pd.read_csv('qualtrics_with_caps.csv', sep = ';')

In [ ]:
with_caps.to_csv('qualtrics_with_caps.csv', sep = ',', index = False)

Below are some steps for getting the PDFs based on the DOIs. Still work in progress. If no DOI is given, use Microsoft Academic (Google Scholar blocks) for a source link -- most of the times it is a direct download link. With a doi, it can be looked up directly on the publishers' websites (works for the major brands). Last resort: using the Sci-Hub API. When the PDFs are saved, the full text can be extracted to do some NLP stuff/clustering etc.

Get PDFs of the ones where I need to look at the full text

In [14]:
import requests
from lxml.html import fromstring
import re, string
pattern = re.compile('[\W_]+')

In [2]:
texts = pd.read_csv('need_full_text.csv')

In [4]:
texts_doi = texts[texts.doi.notnull()]

In [9]:
dois = texts_doi['doi'].tolist()

In [13]:
dois = dois[2:]

In [36]:
dois = dois[2:]

In [39]:
dois

['10.1080/15205436.2018.1472283',
 '10.1080/10714421.2018.1535728',
 '10.1080/19331681.2018.1534703',
 '10.1080/13537121.2018.1505701',
 '10.1177/0163443717734407',
 '10.1515/commun-2018-0012',
 '10.1080/21670811.2017.1377092',
 '10.1080/21670811.2018.1468722',
 '10.1080/15456870.2018.1494176',
 '10.1007/s12027-018-0508-5',
 '10.1016/j.dcm.2018.02.006',
 '10.1080/1461670X.2017.1279028',
 '10.1177/0002764218764251',
 '10.1177/1940161218782740',
 '10.1177/0093650215601883',
 '10.2478/nor-2018-0004',
 '10.1177/1329878X18766078',
 '10.1177/0094582X18757473',
 '10.5210/fm.v23i4.8414',
 '10.1177/1354856516660666',
 '10.1177/1742766517734251',
 '10.1177/1940161218771902',
 '10.1177/0163443717707341',
 '10.1515/commun-2017-0041',
 '10.1177/1077699017721486',
 '10.1109/TKDE.2017.2764084',
 '10.1177/0165551516673285',
 '10.1080/08821127.2017.1419750',
 '10.3233/IP-180072',
 '10.17645/mac.v6i1.1198',
 '10.5209/CLAC.61353',
 '10.1177/0191453717723193',
 '10.1016/j.giq.2017.11.009',
 '10.1080/01449

In [40]:
for item in dois: 
    a = requests.get("http://doi.org/"+item)
    tree = fromstring(a.text)  
    urls = tree.xpath('//a/@href')
    try:
        urls_selection = [s for s in urls if "doi/" in s]
        if urls_selection == []:
            urls_selection = [s for s in urls if "pdf" in s]
        urls_all = [s for s in urls_selection if "http" in s]
        if urls_all == []:
            urls_all = [a.url + s for s in urls_selection]
        pdflink = urls_all[0]
        print(pdflink)
        try:
            res = requests.get(pdflink)
            with open(os.path.join('PDF_full_text/',pattern.sub('', item)), "wb") as f:
                f.write(res.content)
        except:
            print('could not request link for' + item)    
    except:
        print(urls, item)
    

https://www.tandfonline.com/doi/10.1080/10584609.2011.619509
https://www.tandfonline.com/doi/10.1080/1369118X.2015.1109698
https://www.tandfonline.com/doi/10.1080/01972240290075174
https://www.tandfonline.com/doi/10.1080/08997225.2004.10556302
http://journals.sagepub.com/doi/pdf/10.1177/0163443717734407
[] 10.1515/commun-2018-0012
https://www.tandfonline.com/doi/full/10.1080/21670811.2017.1377092/doi/full/10.1080/21670811.2017.1377092?scroll=top&needAccess=true
https://www.tandfonline.com/doi/10.1386/jmpr.13.1.61_1
https://www.tandfonline.com/doi/10.1080/10584600802426965
http://aei.pitt.edu/1156/1/pluralism_gp_COM_92_480.pdf
[] 10.1016/j.dcm.2018.02.006
https://www.tandfonline.com/doi/10.1080/21670811.2013.818365
http://journals.sagepub.com/doi/pdf/10.1177/0002764218764251
http://journals.sagepub.com/doi/pdf/10.1177/1940161218782740
http://journals.sagepub.com/doi/pdf/10.1177/0093650215601883
[] 10.2478/nor-2018-0004
http://journals.sagepub.com/doi/pdf/10.1177/1329878X18766078
http://

ValueError: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.

In [32]:
import os
os.listdir() 

['wos.bib',
 'scopus.bib',
 '.DS_Store',
 'need_full_text.csv',
 'PDF_full_text',
 'qualtrics_with_caps.csv',
 'Conference_Proceedings_list.csv',
 'wos_journals.csv',
 'ebsco_journals.csv',
 'LitReview.ipynb',
 'qualtrics.bib',
 'scopus_journals.csv',
 'ebsco.bib',
 '.ipynb_checkpoints']

In [ ]:
#title = "From Newsworthiness to shareworthiness"
#base_url = 'https://academic.microsoft.com/#/search?'
#rest = '&filters=&from=0&sort=0'
#import urllib
#query = urllib.parse.quote(title)
#full_url = base_url + "iq=%40"+query+"%40&q="+query+rest

In [ ]:
#from selenium import webdriver
#driver = webdriver.PhantomJS()
#driver.get(full_url)
#source = driver.find_element_by_class_name('source-grab').get_attribute('href')
#title = driver.find_element_by_class_name('blue-title').get_attribute('title')
#author = driver.find_element_by_class_name('paper-author-affiliation').text   

In [ ]:
#import requests
#from bs4 import BeautifulSoup

In [ ]:
#for item in with_doi_wos:
#    res = requests.get('https://www.tandfonline.com/doi/pdf/{}'.format(item['doi']))
#    if res.status_code == 404:
#        print(item['doi'])
#        res = requests.get('https://journals.sagepub.com/doi/pdf/{}'.format(item['doi']))
#        if res.status_code == 404:
#            res = requests.get('https://onlinelibrary.wiley.com/doi/pdf/{}'.format(item['doi']))
#            if res.status_code == 404:
#                res = requests.get('https://link.springer.com/content/pdf/{}'.format(item['doi']))
#                if res.status_code == 404:
#                    res = requests.get('https://doi.org/'.format(item['doi']))
#                    soup = BeautifulSoup(res.content, 'html.parser')
#                    try:
#                        link = soup.select('a[name="FullTextPDF"]')
#                        res = requests.get(link[0]['href'])
#                    except:
#                        res = ""
#                        raw = requests.get('https://api.elsevier.com/content/article/doi/{}?httpAccept=text/plain&APIKey=483758170cc6c3159d246d884c526cf0'.format(item['doi']))            
#                        if raw.status_code == 404:
#                            raw = ""
#    if res:
#        with open('Users⁩/felicialoecherbach⁩/surfdrive⁩/literature_review⁩/PDF/{}.pdf'.format(item['ID'].replace(':', ''), 'wb')) as f: 
#            f.write(res.content)
#    if raw:
#        with open('Users⁩/felicialoecherbach⁩/surfdrive⁩/literature_review⁩/RAW/{}.txt'.format(item['ID'].replace(':', ''), 'wb')) as f: 
#            f.write(raw.content)

In [ ]:
#from get_document import Document
#sh = Document()
#sh.download(doi, path = PDFs/{}.pdf'.format(item['ID']))

In [29]:
string = "Users/felicialoecherbach/surfdrive/literature_review/LitReview/PDF_full_text/included/003835@2011.txt"
string[-15:-4]

'003835@2011'

Extract full text from PDFs

In [174]:
import glob, os
import subprocess
os.chdir("/Users/felicialoecherbach1/surfdrive/literature_review/LitReview/PDF_full_text/included")
files = []
for file in glob.glob("*.pdf"):
    files.append(file)
for f in files:
    cmd = 'python3 /Users/felicialoecherbach1/surfdrive/literature_review/pdf2txt.py -o %s.txt %s' % (f.split('.')[0], f)
    run = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = run.communicate()

    # display errors if they occur    
    if err:
        print(err)

KeyboardInterrupt: 

In [232]:
import glob
import errno
text_id = {}
path = '/Users/felicialoecherbach1/surfdrive/literature_review/LitReview/PDF_full_text/included/*.txt'
files = glob.glob(path)
for name in files:
    try:
        with open(name) as f:
            text_id[name[-15:-4]] = f.read()
            pass # do what you want
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise

In [233]:
sentence = text_id['003835@2011']
sentence
sentence_1 = sentence.split('\n\n', 1)[1]
sentence_2 = sentence_1.split('Acknowledgements')[0]
sentence_2

'Article\n\nEuropean Journal of Communication\n26(4) 328 –342\n© The Author(s) 2011 \nReprints and permission: sagepub.\nco.uk/journalsPermissions.nav\nDOI: 10.1177/0267323111423377\nejc.sagepub.com\n\nDo audiences receive diverse \nideas from news media? \nExposure to a variety of \nnews media and personal \ncharacteristics as  \ndeterminants of diversity  \nas received\n\nRichard van der Wurff\nUniversity of Amsterdam, The Netherlands\n\nAbstract\nMedia policies in Europe traditionally promote a diverse media supply. This article investigates \nfor the first time under what conditions audience members actually receive diversity. It focuses on \nthe reception of diverse ideas on European integration from mainstream news media in Ireland \nand the Netherlands. Results show that exposure to a variety of media outlets predicts to some \nextent the reception of diverse ideas, especially for citizens with lower education. But interest \nand  knowledge  are  better  predictors.  This  indic

In [234]:
text_id['001072@2017']

'614244 JOU0010.1177/1464884915614244JournalismRaeijmaekers and Maeseele\n\nresearch-article2015\n\nArticle\n\nIn objectivity we trust? \nPluralism, consensus,  \nand ideology in journalism \nstudies\n\nJournalism\n2017, Vol. 18(6) 647 –663\n© The Author(s) 2015\nReprints and permissions:  \nsagepub.co.uk/journalsPermissions.nav\nDOI: 10.1177/1464884915614244\nhttps://doi.org/10.1177/1464884915614244\njournals.sagepub.com/home/jou\n\nDaniëlle Raeijmaekers\nUniversity of Antwerp, Belgium\n\nPieter Maeseele\nUniversity of Antwerp, Belgium\n\nAbstract\nThis  article  starts  from  the  observation  that  the  concept  of  objectivity,  along  with \nits twin sentries ‘balance’ and ‘impartiality’, is generally regarded as a cornerstone of \njournalism and, consequently, of journalism research. The aim of this article is to show \nthat the analytical ideal of objectivity, instead of enabling, in fact inhibits media pluralism \nresearch. The first section focuses on unveiling the ideological

In [235]:
text_id['003835@2011']

'423377 EJCXXX10.1177/0267323111423377van der Wurff European Journal of Communication\n\nArticle\n\nEuropean Journal of Communication\n26(4) 328 –342\n© The Author(s) 2011 \nReprints and permission: sagepub.\nco.uk/journalsPermissions.nav\nDOI: 10.1177/0267323111423377\nejc.sagepub.com\n\nDo audiences receive diverse \nideas from news media? \nExposure to a variety of \nnews media and personal \ncharacteristics as  \ndeterminants of diversity  \nas received\n\nRichard van der Wurff\nUniversity of Amsterdam, The Netherlands\n\nAbstract\nMedia policies in Europe traditionally promote a diverse media supply. This article investigates \nfor the first time under what conditions audience members actually receive diversity. It focuses on \nthe reception of diverse ideas on European integration from mainstream news media in Ireland \nand the Netherlands. Results show that exposure to a variety of media outlets predicts to some \nextent the reception of diverse ideas, especially for citizens wi

In [236]:
text_id.keys()

dict_keys(['003835@2011', '001072@2017', '004079@2011', '008872@2006', '007928@2007', '005951@2013', '008684@2013', '000926@2017', '005885@1999', '005056@2008', '008237@2011', '001425@2017', '003759@2012', '003650@2012', '008153@2010', '001818@2016', '006759@2017', '002562@2015', '002586@2015', '002003@2016', '002587@2015', '006076@2013', '008152@2011', '004466@2010', '000537@2018', '000063@2018', '001361@2017', '000610@2018', '004801@2009', '001643@2016', '007682@2014', '000149@2018', '001945@2016', '002942@2014', '005337@2006', '004237@2010', '005657@2004', '005701@2002', '007575@2013', '001684@2016', '004240@2010', '002400@2015', '002508@2015', '009007@2007', '005118@2007', '001275@2017', '000562@2018', '001907@2016', '007508@2012', '008685@2017', '000143@2018', '000142@2018', '008471@2017', '007934@2007', '000695@2018', '003631@2012', '004290@2010', '003408@2014', '008578@2016', '001376@2017', '000607@2018', '002230@2015', '004471@2010', '004732@2009', '008311@1999', '002503@2015',

In [10]:
import pandas as pd
import numpy as np
text_ids = pd.DataFrame.from_dict(text_id, orient='index')

NameError: name 'text_id' is not defined

In [238]:
text_ids.to_csv('raw_text.csv')

In [25]:
dataframe = pd.read_csv('/Users/felicialoecherbach1/surfdrive/literature_review/LitReview/PDF_full_text/included/raw_text.csv')

In [7]:
pd.set_option('display.max_colwidth', 100)

In [9]:
coding_raw = pd.read_csv('/Users/felicialoecherbach1/surfdrive/literature_review/LitReview/coding_raw.csv')

In [12]:
coding_raw['Coder'] = np.where(coding_raw['IPAddress']=='146.50.68.108', '1', '2')

In [13]:
without_reliability = coding_raw[coding_raw['Coder'] == '2']

In [14]:
selected = without_reliability[without_reliability['Q3'] == 6]

In [2]:
import os
pdfs_found = [item[:-4] for item in os.listdir("/Users/felicialoecherbach1/surfdrive/literature_review/LitReview/PDF_full_text/included/")]

In [15]:
pdfs_needed = selected['RecipientEmail'].tolist()

In [16]:
len(set(pdfs_found) & set(pdfs_needed))

286

In [17]:
missing = set(pdfs_found).symmetric_difference(set(pdfs_needed))

In [18]:
missing_email = without_reliability[without_reliability['RecipientEmail'].isin(missing)]['RecipientEmail'].tolist()

In [19]:
missing_email

['000159@2018.nl',
 '000291@2018.nl',
 '000653@2018.nl',
 '001398@2017.nl',
 '001430@2017.nl',
 '001689@2016.nl',
 '002827@2014.nl',
 '003394@2014.nl',
 '003772@2012.nl',
 '006378@2013.nl',
 '008804@2004.nl']

In [201]:
missing_email = missing_email[2:]

In [48]:
selected['reason_missing'] = ""

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [74]:
missing = ['000653@2018.nl',
 '001398@2017.nl',
 '001430@2017.nl',
 '001689@2016.nl',
 '002827@2014.nl',
 '003394@2014.nl',
 '003772@2012.nl',
 '006378@2013.nl',
 '008804@2004.nl', 
'006759@2017.nl']

In [75]:
selected['reason_missing'][selected.RecipientEmail.isin(missing)] = "no access"

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:7620: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [76]:
duplicates = ["008820@2007.nl", "008578@2016.nl", '006039@2013.nl', "008507@2018.nl", "008646@2012.nl", "008684@2013.nl", "008631@2014.nl", "008583@2014.nl"]

In [77]:
selected['reason_missing'][selected.RecipientEmail.isin(duplicates)] = "duplicate"

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:7620: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [78]:
selected['reason_missing'].value_counts()

               276
no access       10
duplicate        8
not english      3
Name: reason_missing, dtype: int64

In [67]:
from langdetect import detect

In [68]:
dataframe['language'] = dataframe['0'].apply(detect)

In [69]:
not_english = dataframe[(dataframe['language'].str.contains('es'))|(dataframe['language'].str.contains('pt'))]

In [70]:
not_english = not_english['Unnamed: 0'].tolist()

In [71]:
not_english = [item + ".nl" for item in not_english]

In [72]:
selected['reason_missing'][selected.RecipientEmail.isin(not_english)] = "not english"

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:7620: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [73]:
selected['reason_missing'].value_counts()

               277
no access        9
duplicate        8
not english      3
Name: reason_missing, dtype: int64

In [34]:
ids = selected[selected['reason_missing'] == '']['RecipientEmail']

In [37]:
ids.to_csv('ids2.csv')

In [219]:
dataframe['length'] = dataframe['0'].apply(lambda x: len(x))